In [7]:
library(data.table)
library(dtplyr)
setwd('E:\\kaggle\\market_basket')

In [10]:
orders <- fread('orders.csv')
products <- fread('products.csv')
op_train <- fread('order_products__train.csv')
op_prior <- fread('order_products__prior.csv')
aisles <- fread('aisles.csv')
departments <- fread('departments.csv')

Read 3421083 rows and 7 (of 7) columns from 0.101 GB file in 00:00:03
Read 32434489 rows and 4 (of 4) columns from 0.538 GB file in 00:00:09


In [16]:
orders$order_hour_of_day <- as.numeric(orders$order_hour_of_day)
products$product_name <- as.factor(products$product_name)

In [17]:
dim(orders)
dim(products)
dim(op_train)
dim(op_prior)
dim(aisles)
dim(departments)

[1] 3421083       7

[1] 49688     4

[1] 1384617       4

[1] 32434489        4

[1] 134   2

[1] 21  2

In [18]:
# Number of missing values anywhere in the table
sum(is.na(orders))
sum(is.na(products))
sum(is.na(op_train))
sum(is.na(op_prior))
sum(is.na(aisles))
sum(is.na(departments))

[1] 206209

[1] 0

[1] 0

[1] 0

[1] 0

[1] 0

In [19]:
sapply(orders, function(x) sum(is.na(x)))

order_id                user_id               eval_set 
                     0                      0                      0 
          order_number              order_dow      order_hour_of_day 
                     0                      0                      0 
days_since_prior_order 
                206209

Only days_sine_prior_order column from Orders table have missing values

In [20]:
# Replace missing values to -1
orders[["days_since_prior_order"]][is.na(orders[["days_since_prior_order"]])] <- -1

In [21]:
# combine aisles, departments and products (left joined to products)
#goods = pd.merge(left=pd.merge(left=products, right=departments, how='left'), right=aisles, how='left')
goods = merge(x=merge(products,departments,by = "department_id"), y=aisles, by = "aisle_id") 

In [22]:
dim(goods)

[1] 49688     6

In [23]:
# Merge Orders and Order_products_train data
train_orders = merge(op_train,orders,x.all=TRUE,by = 'order_id')

In [24]:
# Merge Orders and Order_products_prior data
prior_orders = merge(op_prior,orders,x.all = TRUE,by='order_id')

In [25]:
setorderv(prior_orders,"order_number")

In [26]:
prior_orders[user_id==1]

order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
2539329,196,1,0,1,prior,1,2,8,-1
2539329,14084,2,0,1,prior,1,2,8,-1
2539329,12427,3,0,1,prior,1,2,8,-1
2539329,26088,4,0,1,prior,1,2,8,-1
2539329,26405,5,0,1,prior,1,2,8,-1
2398795,196,1,1,1,prior,2,3,7,15
2398795,10258,2,0,1,prior,2,3,7,15
2398795,12427,3,1,1,prior,2,3,7,15
2398795,13176,4,0,1,prior,2,3,7,15
2398795,26088,5,1,1,prior,2,3,7,15


In [27]:
setorderv(train_orders,"order_number")

In [28]:
train_orders[user_id==1]

order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1187899,196,1,1,1,train,11,4,8,14
1187899,25133,2,1,1,train,11,4,8,14
1187899,38928,3,1,1,train,11,4,8,14
1187899,26405,4,1,1,train,11,4,8,14
1187899,39657,5,1,1,train,11,4,8,14
1187899,10258,6,1,1,train,11,4,8,14
1187899,13032,7,1,1,train,11,4,8,14
1187899,26088,8,1,1,train,11,4,8,14
1187899,27845,9,0,1,train,11,4,8,14
1187899,49235,10,1,1,train,11,4,8,14


We See that, for user_id = 1, 10 of the prior orders are present in prior_order dataset and the 11th order in train data. Let's combine these two to get our train dataset.

Also,

Reordered colomn will tell us if the product is begin ordered for the first time or not.

order number will give us the sequence of order

add_to_cart_order gives us the sequence by which the product is added to cart

In [29]:
#prior_in_train = prior_orders[prior_orders.user_id.isin(train_orders.user_id.unique())]
prior_in_train = prior_orders[user_id %in% unique(train_orders$user_id)]

In [30]:
dim(prior_in_train)
length(unique(prior_orders$user_id))
length(unique(train_orders$user_id))
length(unique(prior_in_train$user_id))

[1] 20641991       10

[1] 206209

[1] 131209

[1] 131209

In [31]:
# append prior_in_train and train_orders
train_prior_orders <- rbindlist(list(prior_in_train,train_orders),use.names=TRUE)

In [32]:
# Merge goods data to train dataset
train <- merge(train_prior_orders, goods, x.all = TRUE, by = 'product_id')

In [33]:
dim(train)

[1] 22026608       15

In [34]:
head(train)

product_id,order_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id,department_id,product_name,department,aisle
1,9273,30,0,50005,prior,1,1,15,-1,61,19,Chocolate Sandwich Cookies,snacks,cookies cakes
1,114795,8,0,92648,prior,1,3,15,-1,61,19,Chocolate Sandwich Cookies,snacks,cookies cakes
1,119302,4,0,108456,prior,1,1,9,-1,61,19,Chocolate Sandwich Cookies,snacks,cookies cakes
1,207661,2,0,3904,prior,1,1,14,-1,61,19,Chocolate Sandwich Cookies,snacks,cookies cakes
1,354088,7,0,122586,prior,1,2,8,-1,61,19,Chocolate Sandwich Cookies,snacks,cookies cakes
1,361190,1,0,85811,prior,1,4,8,-1,61,19,Chocolate Sandwich Cookies,snacks,cookies cakes


In [40]:
# perpare test data
test_orders <- orders[eval_set=='test']

In [41]:
prior_in_test = prior_orders[user_id %in% unique(test_orders$user_id)]